# Dataset

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
import torch
from transformers import T5Tokenizer
import numpy as np


/Users/apple/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%%capture

tokenizer = T5Tokenizer.from_pretrained('t5-small', padding=True)

def _tokenize( x):
    return tokenizer(x, return_tensors="pt")['input_ids'][0][:-1]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# !pip install sentencepiece

## Hop1Index

In [5]:

class Hop1Index:
    def __init__(self, triples, num_entities, key_col=0, max_context_size=64):
        self.max_context_size = max_context_size
        self.shuffle = False
        self.key_col = key_col
        self.triples = triples[triples[:, key_col].argsort()]
        keys, values_offset = np.unique(
            self.triples[:, key_col], axis=0, return_index=True
        )
        values_offset = np.append(values_offset, len(self.triples))
        self.keys = keys
        self.values_offset = values_offset

        self.key_to_start = -1 * np.ones(num_entities, dtype=int)
        self.key_to_start[keys] = values_offset[:-1]
        self.key_to_end = -1 * np.ones(num_entities, dtype=int)
        self.key_to_end[keys] = values_offset[1:]

    def __getitem__(self, item, rel_id=None):
        start = self.key_to_start[item]
        end = self.key_to_end[item]
        context = self.triples[start:end, [1, 2 - self.key_col]]
        if rel_id is not None:
            context = context[context[:,0] == rel_id][:,1]
        if len(context) > self.max_context_size:
            ids = np.random.choice(len(context), self.max_context_size, replace=False)
            context = context[ids]
        if self.shuffle:
            np.random.shuffle(context)
        return context

    def get_context(self, item, rel_id=None):
        return self.__getitem__(item, rel_id)



## KGCDataset

In [6]:
kgt5_data = torch.load('kg_data.pt')

In [7]:
from numpy import pi

class RotatE:
    def __init__(self, k, max_rel_size=None, entity_embedding=None, relation_embedding=None):
        self.internal_k = 2 * k
        self.max_rel_size = max_rel_size
        self.entity_embedding = entity_embedding
        self.relation_embedding = relation_embedding

    def __call__(self, e_s_id, e_p_id):
        e_s = self.entity_embedding[e_s_id]
        e_p = self.relation_embedding[e_p_id]
        e_s_real, e_s_img = torch.chunk(e_s, 2, axis=0)
        theta_pred, _ = torch.chunk(e_p, 2, axis=0)

        embedding_range = (6 / (self.internal_k * self.max_rel_size)) ** 0.5
        e_p_real = torch.cos(theta_pred / (embedding_range / pi))
        e_p_img = torch.sin(theta_pred / (embedding_range / pi))

        e_o_real = e_s_real * e_p_real - e_s_img * e_p_img
        e_o_img = e_s_real * e_p_img + e_s_img * e_p_real
        return torch.cat([e_o_real, e_o_img], axis=0)

rotatE = RotatE(k=350, entity_embedding=kgt5_data['RotatE_ent_emb'], relation_embedding=kgt5_data['RotatE_rel_emb'], max_rel_size=237)


In [8]:
import numpy as np
from torch.utils.data import Dataset
from typing import Dict, Optional, Union, Tuple, List
import random


class KGCDataset(Dataset):
    def __init__(self, num_ents=14541, structal_model=None):
        self.num_ents = num_ents
        self.structal_model = structal_model
        # Fb15k wn18rr
        self.id_triplets ={
            'train': kgt5_data['train_triplet_id'],
            'valid': kgt5_data['valid_triplet_id'],
            'test': kgt5_data['test_triplet_id']
        }
        self.tokens_triplets ={
            'train': kgt5_data['train_triplet_tokens'],
            'valid': kgt5_data['valid_triplet_tokens'],
            'test': kgt5_data['test_triplet_tokens']
        }
        self.decs_triplets ={
            'train': kgt5_data['train_triplet_decs'],
            'valid': kgt5_data['valid_triplet_decs'],
            'test': kgt5_data['test_triplet_decs']
        }

        self.get_neigs_0 ={
            'train': Hop1Index(self.id_triplets['train'], self.num_ents, 0),
            'valid': Hop1Index(self.id_triplets['train'],self.num_ents, 0),
            'test': Hop1Index(self.id_triplets['train'],self.num_ents, 0)
        }
        self.get_neigs_2 ={
            'train': Hop1Index(self.id_triplets['train'], self.num_ents, 2),
            'valid': Hop1Index(self.id_triplets['train'],self.num_ents, 2),
            'test': Hop1Index(self.id_triplets['train'],self.num_ents, 2)
        }

        self.mask_token = _tokenize('<extra_id_90>')
        self.eos_token = torch.tensor([tokenizer.eos_token_id])
        self.zero_neig_embedding = torch.zeros([512])

        self.predict_head_token = _tokenize('predict head :')
        self.predict_tail_token = _tokenize('predict tail :')
        self.start_decs_token = _tokenize('[')
        self.end_decs_token = _tokenize(']')
        self.inversion_token = _tokenize('inversion of ')
        self.empty_token = torch.tensor([], dtype=torch.int)
        self.set_ent_id = set(range(self.num_ents))
        self.p_dropout = 0. # 0.2 when training

    def __getitem__(self, idx):
        return self.get(idx, split=self.split)
    def __len__(self, split='train'):
        return len(self.tokens_triplets[split])

    def get(self, idx: int, split: str = "train", full_mask_part_idx=None):
        head_lbl, relation, tail_lbl = self.tokens_triplets[split][idx]
        head_id, rel_id, tail_id = self.id_triplets[split][idx]
        head_decs, tail_decs = self.decs_triplets[split][idx]

        if full_mask_part_idx is None:
          full_mask_part_idx = 2 if random.randint(0, 1) else 0

        inversion = False

        if full_mask_part_idx:
          source = [
              self.predict_tail_token if not inversion else self.predict_head_token,
              head_lbl,
              self.start_decs_token,
              head_decs,
              self.end_decs_token,
              self.inversion_token if inversion else self.empty_token,
              relation,
          ]
          target = [tail_lbl]
          # filter_id = torch.cat([set_neig.get_context(head_id, rel_id) for set_neig in self.get_neigs_0.values()])
          neighboors_0 = self.get_neigs_0[split][head_id]
          neighboors_0 = neighboors_0[(neighboors_0[:,0]!=rel_id) | (neighboors_0[:,1]!=tail_id)]
          neighboors_2 = self.get_neigs_2[split][head_id]
          neighboors_2 = neighboors_2[(neighboors_2[:,0]!=rel_id) | (neighboors_2[:,1]!=tail_id)]
        else:
          source = [
              self.predict_head_token if not inversion else self.predict_tail_token,
              tail_lbl,
              self.start_decs_token,
              tail_decs,
              self.end_decs_token,
              self.inversion_token if inversion else self.empty_token,
              relation,
          ]
          target = [head_lbl]
          # filter_id = torch.cat([set_neig.get_context(tail_id, rel_id) for set_neig in self.get_neigs_2.values()])
          neighboors_0 = self.get_neigs_0[split][tail_id]
          neighboors_0 = neighboors_0[(neighboors_0[:,0]!=rel_id) | (neighboors_0[:,1]!=head_id)]
          neighboors_2 = self.get_neigs_2[split][tail_id]
          neighboors_2 = neighboors_2[(neighboors_2[:,0]!=rel_id) | (neighboors_2[:,1]!=head_id)]

        target_ent_embeddings = []
        neighboors_embeddings = []
        for rel_n_id, ent_n_id in neighboors_0:
          if ent_n_id >= 14505:
            continue
          ent_n_embedding = self.structal_model.entity_embedding[ent_n_id]
          rel_n_embedding = self.structal_model.relation_embedding[rel_n_id]
          target_ent_embedding = self.structal_model(ent_n_id, rel_n_id)
          neighboors_embeddings.append(torch.cat([ent_n_embedding, rel_n_embedding]))
          target_ent_embeddings.append(target_ent_embedding)
        for rel_n_id, ent_n_id in neighboors_2:
          if ent_n_id >= 14505:
            continue
          ent_n_embedding = self.structal_model.entity_embedding[ent_n_id]
          rel_n_embedding = self.structal_model.relation_embedding[rel_n_id]
          target_ent_embedding = self.structal_model(ent_n_id, rel_n_id)
          neighboors_embeddings.append(torch.cat([ent_n_embedding, -rel_n_embedding]))
          target_ent_embeddings.append(target_ent_embedding)

        if len(neighboors_embeddings):
          neighboors_embeddings = torch.stack(neighboors_embeddings)
          target_ent_embeddings = torch.stack(target_ent_embeddings)
          neighboors_embeddings_mask = torch.ones(len(neighboors_embeddings))
        else:
          neighboors_embeddings_mask = torch.zeros([1])
          neighboors_embeddings = torch.zeros([1, 700*2])
          target_ent_embeddings = torch.zeros([1, 700])


        source.append(self.eos_token)
        target.append(self.eos_token)
        source = torch.cat(source)
        target = torch.cat(target)

        # seq2seq attention dropout
        attention_mask = torch.ones_like(source)
        rand = torch.rand_like(attention_mask.float())
        dropout = torch.logical_not(rand < self.p_dropout).long()
        dropout[(source == self.start_decs_token[0]) | (source == self.end_decs_token[0])] = 1
        dropout[:4]=1
        inversion_len = len(self.inversion_token if inversion else self.empty_token)
        relation_len = len(relation)
        dropout[-relation_len-inversion_len:-relation_len]=1
        attention_mask = attention_mask * dropout


        output = {
            "input_ids": source,
            "attention_mask": attention_mask,
            "labels": target,
            'neighboors_embeddings': neighboors_embeddings,
            'neighboors_embeddings_mask': neighboors_embeddings_mask,
            'target_ent_embeddings': target_ent_embeddings,
            'triplet': self.id_triplets[split][idx],
            'neighboors_0_id': neighboors_0,
            'neighboors_2_id': neighboors_2,
        }
        return output

dataset = KGCDataset(num_ents=14541, structal_model=rotatE)

ext_get_neigs_0 ={
    'train': Hop1Index(
        kgt5_data['train_triplet_id'],
        dataset.num_ents, 0, max_context_size=1e10),
    'valid': Hop1Index(
        kgt5_data['valid_triplet_id'],
        dataset.num_ents, 0, max_context_size=1e10),
    'test': Hop1Index(
        kgt5_data['test_triplet_id'],
        dataset.num_ents, 0, max_context_size=1e10),
}

ext_get_neigs_2 ={
    'train': Hop1Index(
        kgt5_data['train_triplet_id'],
        dataset.num_ents, 2, max_context_size=1e10),
    'valid': Hop1Index(
        kgt5_data['valid_triplet_id'],
        dataset.num_ents, 2, max_context_size=1e10),
    'test': Hop1Index(
        kgt5_data['test_triplet_id'],
        dataset.num_ents, 2, max_context_size=1e10),
}

# get all ground truth
def get_neigs2(ent_id, rel_id):
  n_train = ext_get_neigs_2['train'].__getitem__(ent_id, rel_id)
  n_valid = ext_get_neigs_2['valid'].__getitem__(ent_id, rel_id)
  n_test = ext_get_neigs_2['test'].__getitem__(ent_id, rel_id)
  return [n_train, n_valid, n_test]
# get all ground truth
def get_neigs0(ent_id, rel_id):
  n_train = ext_get_neigs_0['train'].__getitem__(ent_id, rel_id)
  n_valid = ext_get_neigs_0['valid'].__getitem__(ent_id, rel_id)
  n_test = ext_get_neigs_0['test'].__getitem__(ent_id, rel_id)
  return [ n_train, n_valid, n_test]

class SplitDatasetWrapper:
    def __init__(self, dataset, split, full_mask_part_idx=None):
        self.dataset = dataset
        self.split = split
        self.full_mask_part_idx = full_mask_part_idx
    def __getitem__(self, idx):
        return self.dataset.get(idx, self.split, self.full_mask_part_idx)
    def __len__(self):
        return self.dataset.__len__(split=self.split)

train_dataset = SplitDatasetWrapper(dataset, split="train")
valid_dataset = SplitDatasetWrapper(dataset, split="valid")
test_dataset = SplitDatasetWrapper(dataset, split="test")

head_test_dataset = SplitDatasetWrapper(dataset, split="test", full_mask_part_idx=0)
tail_test_dataset = SplitDatasetWrapper(dataset, split="test", full_mask_part_idx=2)

head_valid_dataset = SplitDatasetWrapper(dataset, split="valid", full_mask_part_idx=0)
tail_valid_dataset = SplitDatasetWrapper(dataset, split="valid", full_mask_part_idx=2)

head_train_dataset = SplitDatasetWrapper(dataset, split="train", full_mask_part_idx=0)
tail_train_dataset = SplitDatasetWrapper(dataset, split="train", full_mask_part_idx=2)

# model

In [9]:
from src.model import StructKS2S

model_name='t5-small'
model = StructKS2S.from_pretrained(model_name)
model_state_dict = torch.load('models/saved_models/structkgs2s_fb15k237.pt', map_location='cpu')
model.load_state_dict(model_state_dict)



/Users/apple/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of StructKS2S were not initialized from the model checkpoint at t5-small and are newly initialized: ['key_projection.bias', 'key_projection.weight', 'value_projection.bias', 'value_projection.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [10]:
# !pip install transformers==4.40.1



## DataCollatorForSeq2Seq

In [11]:
# from torch.nn.utils.rnn import pad_sequence

# class DataCollatorForSeq2Seq:
#     model= None
#     padding= True
#     max_length= None
#     pad_to_multiple_of=None
#     label_pad_token_id= -100
#     data_names = None
#     def __init__(self, tokenizer, model=None, padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100,data_names=None):
#         self.tokenizer = tokenizer
#         self.model = model
#         self.data_names = data_names
#         self.label_pad_token_id = label_pad_token_id

#     def __call__(self, features):
#         features2 = {}
#         for name in self.data_names:
#           if name == 'triplet':
#             continue
#           if name in ['labels','filter_id']:
#             padding_value=self.label_pad_token_id
#           else:
#             padding_value=self.tokenizer.pad_token_id
#           x_features = [feature[name] for feature in features]
#           features2[name] = torch.nn.utils.rnn.pad_sequence(x_features, batch_first=True, padding_value=padding_value)
#         if self.model is not None and hasattr(self.model, "prepare_decoder_input_ids_from_labels"):
#             decoder_input_ids = self.model.prepare_decoder_input_ids_from_labels(labels=features2["labels"])
#             features2["decoder_input_ids"] = decoder_input_ids
#         return features2


# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, data_names=list(train_dataset[0].keys()))


In [12]:
# data = data_collator([train_dataset[0]])

In [13]:
# model(**data)

# train

In [14]:
from transformers import Seq2SeqTrainingArguments, TrainingArguments
from transformers import Seq2SeqTrainer
batch_size= 32*4

args = Seq2SeqTrainingArguments(
    "kgt5-rotatE",
    dataloader_num_workers=8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    num_train_epochs=100,
    do_eval=True,

    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy='epoch',

    learning_rate=1e-4,
    # torch_compile=True,
    fp16=True,

    tf32=True,
    report_to='none',
    load_best_model_at_end=True,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
from transformers import Seq2SeqTrainer
from transformers import EarlyStoppingCallback

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
)

In [ ]:
!gsutil cp /content/kgt5-rotatE/checkpoint-25512.zip gs://hien7613storage2/

Copying file:///content/kgt5-rotatE/checkpoint-25512.zip [Content-Type=application/zip]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][618.6 MiB/618.6 MiB]                                                
Operation completed over 1 objects/618.6 MiB.                                    


In [ ]:
trainer.train(resume_from_checkpoint='/content/kgt5-rotatE/checkpoint-12756')
# baaed1dc0ef02b02dff291c8e0cfacf571bff2f9

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Pleas

Epoch,Training Loss,Validation Loss
7,1.785700,1.514204
8,1.719100,1.452341
9,1.655400,1.419580
10,1.609600,1.393566
11,1.570200,1.364775
12,1.538500,1.371552
13,3.801800,4.198689
14,5.522400,4.242440


KeyboardInterrupt: 

In [ ]:
trainer.train()
# baaed1dc0ef02b02dff291c8e0cfacf571bff2f9

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.996500,2.332599
2,2.492400,2.038121
3,2.262600,1.857196
4,2.093300,1.739745
5,1.968800,1.647498
6,1.867400,1.564553
7,1.785700,1.514724
8,1.719600,1.453535
9,1.656200,1.420135
10,1.609800,1.395179


KeyboardInterrupt: 

In [1]:
# import torch

# trainer.model.eval()
# model_state_dict = trainer.model.state_dict()
# torch.save(model_state_dict, '/content/kgt5_rotatE_x12.pt')

# !gsutil -o GSUtil:parallel_composite_upload_threshold=150M cp  /content/kgt5_rotatE_x12.pt gs://hien7613storage2/

# New Eval

## setup eval

In [15]:
# ent2text
from tqdm.auto import tqdm




path = "data/raw/fb15k-237/entity2text.txt"

ent2text = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    ent, text = line.strip().split('\t')
    ent2text[ent] = _tokenize(text)
    
path = "data/raw/fb15k-237/relation2text.txt"

rel2text = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    rel, text = line.strip().split('\t')
    rel2text[rel] = _tokenize(text)

# ent2id
path = "data/raw/fb15k-237/entities.txt"

ent2id = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    # print(line.strip().split('\t'))
    ent = line.strip().split('\t')[0]
    ent2id[ent] = int(i)
    
# rel2id
path = "data/raw/fb15k-237/relations.txt"

rel2id = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    rel = line.strip().split('\t')[0]
    rel2id[rel] = int(i)

entid2text = [0]*len(ent2id)
for ent in tqdm(ent2id):
  entid2text[ent2id[ent]] = [0] + ent2text[ent].tolist() + [1]
  
relid2text = [0]*len(rel2id)
for rel in tqdm(rel2id):
  relid2text[rel2id[rel]] = [0] + rel2text[rel].tolist() + [1]

ent_name_decode_list = []
for target in tqdm(entid2text):
  ent_name_decode_list.append(tokenizer.decode(target[1:-1]))

rel_name_decode_list = []
for target in tqdm(relid2text):
  rel_name_decode_list.append(tokenizer.decode(target[1:-1]))

Processing lines:   0%|          | 0/14951 [00:00<?, ?it/s]

100%|██████████| 237/237 [00:00<00:00, 2575.95it/s]


In [16]:
from typing import Dict, List
class Trie(object):
    def __init__(self, sequences: List[List[int]] = []):
        self.trie_dict = {}
        self.len = 0
        if sequences:
            for sequence in sequences:
                Trie._add_to_trie(sequence, self.trie_dict)
                self.len += 1
        self.append_trie = None
        self.bos_token_id = None
    def append(self, trie, bos_token_id):
        self.append_trie = trie
        self.bos_token_id = bos_token_id
    def add(self, sequence: List[int]):
        Trie._add_to_trie(sequence, self.trie_dict)
        self.len += 1
    def get(self, prefix_sequence: List[int]):
        return Trie._get_from_trie(prefix_sequence, self.trie_dict, self.append_trie, self.bos_token_id)
    @staticmethod
    def load_from_dict(trie_dict):
        trie = Trie()
        trie.trie_dict = trie_dict
        trie.len = sum(1 for _ in trie)
        return trie
    @staticmethod
    def _add_to_trie(sequence: List[int], trie_dict: Dict):
        if sequence:
            if sequence[0] not in trie_dict:
                trie_dict[sequence[0]] = {}
            Trie._add_to_trie(sequence[1:], trie_dict[sequence[0]])
    @staticmethod
    def _get_from_trie(
        prefix_sequence: List[int],
        trie_dict: Dict,
        append_trie=None,
        bos_token_id: int = None,
    ):
        if len(prefix_sequence) == 0:
            output = list(trie_dict.keys())
            if append_trie and bos_token_id in output:
                output.remove(bos_token_id)
                output += list(append_trie.trie_dict.keys())
            if len(output) == 0:
                return [0]
            return output
        elif prefix_sequence[0] in trie_dict:
            return Trie._get_from_trie(
                prefix_sequence[1:],
                trie_dict[prefix_sequence[0]],
                append_trie,
                bos_token_id,
            )
        else:
            if append_trie:
                return append_trie.get(prefix_sequence)
            else:
                return [0]
    def __iter__(self):
        def _traverse(prefix_sequence, trie_dict):
            if trie_dict:
                for next_token in trie_dict:
                    yield from _traverse(prefix_sequence + [next_token], trie_dict[next_token])
            else:
                yield prefix_sequence

        return _traverse([], self.trie_dict)
    def __len__(self):
        return self.len
    def __getitem__(self, value):
        return self.get(value)
trie = Trie(entid2text)
# trie = kgt5_data2['trie']

In [17]:
import numpy as np
import pandas as pd
def _get_performance(ranks):
    ranks = np.array(ranks, dtype=np.float32)
    out = dict()
    out['mr'] = ranks.mean(axis=0)
    out['mrr'] = (1. / ranks).mean(axis=0)
    out['hit1'] = np.sum(ranks == 1, axis=0) / len(ranks)
    out['hit3'] = np.sum(ranks <= 3, axis=0) / len(ranks)
    out['hit10'] = np.sum(ranks <= 10, axis=0) / len(ranks)
    return out


def get_performance(model, tail_ranks, head_ranks):
    tail_out = _get_performance(tail_ranks)
    head_out = _get_performance(head_ranks)
    mr = np.array([tail_out['mr'], head_out['mr']])
    mrr = np.array([tail_out['mrr'], head_out['mrr']])
    hit1 = np.array([tail_out['hit1'], head_out['hit1']])
    hit3 = np.array([tail_out['hit3'], head_out['hit3']])
    hit10 = np.array([tail_out['hit10'], head_out['hit10']])
    perf = {'mrr': mrr, 'mr': mr, 'hit@1': hit1, 'hit@3': hit3, 'hit@10': hit10}
    perf = pd.DataFrame(perf, index=['tail ranking', 'head ranking'])
    perf.loc['mean ranking'] = perf.mean(axis=0)
    for hit in ['hit@1', 'hit@3', 'hit@5', 'hit@10']:
        if hit in list(perf.columns):
            perf[hit] = perf[hit].apply(lambda x: '%.2f%%' % (x * 100))
    return perf



In [18]:
list_global_ranks = []

def compute_metrics(ranks):
    mrr = sum(1.0 / rank for rank in ranks) / len(ranks)
    hit1 = sum(1 for rank in ranks if rank <= 1) / len(ranks)
    hit5 = sum(1 for rank in ranks if rank <= 5) / len(ranks)
    hit10 = sum(1 for rank in ranks if rank <= 10) / len(ranks)
    
    return mrr, hit1, hit5, hit10
    
class RunEval:
    def __init__(self, configs, model, tokenizer, ent_name_list, target_embeddings, device='mps'):
        self.configs = configs
        self.ent_name_list = ent_name_list
        self.target_embeddings = target_embeddings
        self.configs = configs
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.model.to(device)
        self.model.eval()



    def _next_candidate(self, batch_idx, input_ids, triple_id, dataset_idx, old_seqs=None):
        input_ids = input_ids.cpu()

        if input_ids[-1] == 0 and len(input_ids) != 1:
            return [0]
        pred_ids = self.target_embeddings[triple_id[batch_idx][dataset_idx]]
        pred_id = int(pred_ids[len(input_ids)])
        all_gt_ids = torch.cat(self.get_neigs(triple_id[batch_idx][2-dataset_idx], triple_id[batch_idx][1]))

        all_gt_seq = torch.index_select(self.target_embeddings, 0, all_gt_ids)
        all_gt_seq_mask = (all_gt_seq[:, :len(input_ids)]==input_ids).all(1)
        all_gt_seq_tokens = all_gt_seq[:, len(input_ids)][all_gt_seq_mask]
        if len(old_seqs) > 0:
          old_seq = torch.nn.utils.rnn.pad_sequence([x[batch_idx] for x in old_seqs], batch_first=True, padding_value=0)
          if old_seq.shape[1] > len(input_ids):
            old_seq_mask = (old_seq[:, :len(input_ids)]==input_ids).all(1)
            old_seq_tokens = old_seq[:, len(input_ids)][old_seq_mask]
          else:
            old_seq_tokens = torch.tensor([], dtype=torch.int64)
        else:
          old_seq_tokens = torch.tensor([], dtype=torch.int64)
        all_gt_seq_tokens = set(torch.cat([all_gt_seq_tokens, old_seq_tokens]).tolist())
        pred_id = int(pred_ids[len(input_ids)])
        next_tokens = set(trie.get(input_ids.tolist())).difference(all_gt_seq_tokens)
        if pred_id in all_gt_seq_tokens:
          next_tokens.add(pred_id)
        if len(next_tokens) == 0:
          return [0]
        next_tokens = next_tokens - set( old_seq_tokens.tolist())
        return list(next_tokens)

    def validation_epoch_end(self, outs):
        pred_tail_out, pred_head_out = outs
        agg_tail_out, agg_head_out = dict(), dict()
        for out in pred_tail_out:
            for key, value in out.items():
                if key in agg_tail_out:
                    agg_tail_out[key] += value
                else:
                    agg_tail_out[key] = value
        for out in pred_head_out:
            for key, value in out.items():
                if key in agg_head_out:
                    agg_head_out[key] += value
                else:
                    agg_head_out[key] = value
        tail_ranks, head_ranks = agg_tail_out['ranks'], agg_head_out['ranks']
        del agg_tail_out['ranks']
        del agg_head_out['ranks']
        perf = get_performance(self, head_ranks, tail_ranks)
        print(perf)
        return perf




    @torch.no_grad()
    def validation_step(self, batched_data, dataset_idx):
        global list_global_ranks
        input_ids = batched_data['input_ids'].to(self.device)
        attention_mask = batched_data['attention_mask'].to(self.device)
        labels = batched_data['labels']
        labels = torch.where(labels != -100, labels, self.tokenizer.pad_token_id)
        neighboors_embeddings=batched_data['neighboors_embeddings'].to(self.device)
        neighboors_embeddings_mask=batched_data['neighboors_embeddings_mask'].to(self.device)
        target_ent_embeddings=batched_data['target_ent_embeddings'].to(self.device)
        neighboors_0 = batched_data['neighboors_0_id']
        neighboors_2 = batched_data['neighboors_2_id']
        triple_id = batched_data['triplet'].numpy()

        self.get_neigs = get_neigs2 if dataset_idx == 0 else get_neigs0

        old_seqs = []
        
        list_pred_texts = [[0]*self.configs.num_beams for _ in range(len(labels))]
        
        ranks = torch.randint(self.configs.num_beams + 1, self.configs.n_ent, (len(labels),))
        for i in range(self.configs.num_beams):
          outputs = self.model.generate(
              input_ids=input_ids,
              attention_mask=attention_mask,
              return_dict_in_generate=True,
              max_length=512,
              prefix_allowed_tokens_fn=lambda batch_idx, m_input_ids: self._next_candidate(batch_idx, m_input_ids, triple_id, dataset_idx, old_seqs),
              neighboors_embeddings=neighboors_embeddings,
              neighboors_embeddings_mask=neighboors_embeddings_mask,
              target_ent_embeddings=target_ent_embeddings,
          )
          pred = outputs.sequences.cpu()
          old_seqs.append(pred)
          pred = pred[:,1:]          
          seq_len = min(pred.shape[1], labels.shape[1])
          pred = pred[:, :seq_len]
          cut_labels = labels[:, :seq_len]
          seq_match = (pred == cut_labels).all(1)
          new_ranks = torch.where(~seq_match, ranks, i+1)
          ranks = torch.min(ranks, new_ranks)
          pred_texts = [self.tokenizer.decode(x, skip_special_tokens = True) for x in pred]
          for j, pred_text in enumerate(pred_texts):
            list_pred_texts[j][i] = pred_text
          
        list_input_texts = [self.tokenizer.decode(x, skip_special_tokens = True) for x in input_ids]
        list_target_texts = [self.tokenizer.decode(x, skip_special_tokens = True) for x in labels]
        for i in range(len(list_input_texts)):
          rank = ranks[i].item()
          print(f'Input: {list_input_texts[i]}')
          relation_name = rel_name_decode_list[triple_id[i][1]]
          print(f'Relation: {relation_name}')
          print('Neighbors:')
          for n in neighboors_0[i][:5]:
            rel_id = n[0]
            ent_id = n[1]
            if rel_id == 0 and ent_id == 0:
              continue
            rel_name = rel_name_decode_list[rel_id]
            ent_name = self.ent_name_list[ent_id]
            if '.' in rel_name:
              rel_name = rel_name.split('.')[1].strip()
            print(f'--{rel_name}--> {ent_name}')
          for n in neighboors_2[i][:5]:
            rel_id = n[0]
            ent_id = n[1]
            if rel_id == 0 and ent_id == 0:
              continue
            rel_name = rel_name_decode_list[rel_id]
            ent_name = self.ent_name_list[ent_id]
            if '.' in rel_name:
              rel_name = rel_name.split('.')[1].strip()
            print(f'--{rel_name}--> {ent_name}')
          print('...') 

          print(f'Target: {list_target_texts[i]}')
          print(f'Predictions: {list_pred_texts[i]}')
          print(f'Rank: {rank if rank <= self.configs.num_beams else str(rank) + "(Out of beams)"}')
          print('='*100)
          list_global_ranks.append(rank)
          
        mrr, hit1, hit5, hit10 = compute_metrics(list_global_ranks)
        print(f'MRR: {mrr:.4f}, Hit@1: {hit1:.4f}, Hit@5: {hit5:.4f}, Hit@10: {hit10:.4f}')
        print('='*500)


        ranks = ranks.tolist()
        out = {'ranks': ranks}
        return out

In [19]:
entid2text_emb = torch.nn.utils.rnn.pad_sequence([torch.tensor(x) for x in entid2text], batch_first=True, padding_value=0)

## run eval

In [20]:

class DataCollatorForSeq2Seq:
    model= None
    padding= True
    max_length= None
    pad_to_multiple_of=None
    label_pad_token_id= -100
    data_names = None
    def __init__(self, tokenizer, model=None, padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100,data_names=None):
        self.tokenizer = tokenizer
        self.model = model
        self.data_names = data_names
        self.label_pad_token_id = label_pad_token_id

    def __call__(self, features):
        features2 = {}
        for name in self.data_names:
          if name in ['labels','filter_id']:
            padding_value=self.label_pad_token_id
          else:
            padding_value=self.tokenizer.pad_token_id
          x_features = [feature[name] for feature in features]
          if name in ['neighboors_0', 'neighboors_2']:
            features2[name] = x_features
          else:
            features2[name] = torch.nn.utils.rnn.pad_sequence(x_features, batch_first=True, padding_value=padding_value)
        if self.model is not None and hasattr(self.model, "prepare_decoder_input_ids_from_labels"):
            decoder_input_ids = self.model.prepare_decoder_input_ids_from_labels(labels=features2["labels"])
            features2["decoder_input_ids"] = decoder_input_ids
        return features2


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, data_names=list(train_dataset[0].keys()))


In [21]:
# tail_test_dataset[9]

In [ ]:
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

batch_size = 16
tail_data_loader = DataLoader(tail_test_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          collate_fn=data_collator,
                          # num_workers=8,
                          pin_memory=True)
head_data_loader = DataLoader(head_test_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          # num_workers=8,
                          collate_fn=data_collator,
                          pin_memory=True)

class Configs:
    def __init__(self):
        self.num_beams = 10
        self.num_return_sequences = 10
        self.max_length = 30    
        self.n_ent = 14541
        self.n_rel = 237

configs = Configs()
runEval = RunEval(configs, model, tokenizer, ent_name_decode_list, entid2text_emb)

runEval.model.eval()
for batch1, batch2 in tqdm(zip(head_data_loader, tail_data_loader), total=len(tail_data_loader)):
    runEval.validation_step(batch1, 0)
    runEval.validation_step(batch2, 2)


  0%|          | 0/1280 [00:00<?, ?it/s]

In [ ]:
# %pip install transformers==4.40.1

In [ ]:
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

tail_data_loader = DataLoader(tail_test_dataset,
                          batch_size=64,
                          shuffle=False,
                          collate_fn=data_collator,
                          # num_workers=8,
                          pin_memory=True)
head_data_loader = DataLoader(head_test_dataset,
                          batch_size=64,
                          shuffle=False,
                          # num_workers=8,
                          collate_fn=data_collator,
                          pin_memory=True)


class Configs:
    def __init__(self):
        self.num_beams = 1
        # self.num_beam_groups = 1
        self.num_return_sequences = 1
        self.max_length = 30
        self.n_ent = 14541
        self.n_rel = 237

configs = Configs()
runEval = RunEval(configs, model, tokenizer, ent_name_decode_list, entid2text_emb)

runEval.model.eval()
head_list_result = []
for data in tqdm(head_data_loader):
    # pass
    rank_rs = runEval.validation_step(data, 0)
    head_list_result.append(rank_rs)
    # break
tail_list_result = []
for data in tqdm(tail_data_loader):
    rank_rs = runEval.validation_step(data, 2)
    tail_list_result.append(rank_rs)
    # break

kq = runEval.validation_epoch_end((head_list_result, tail_list_result))


  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

                   mrr           mr   hit@1   hit@3  hit@10
tail ranking  0.479574  2367.001953  39.12%  53.62%  67.29%
head ranking  0.353449  3125.134521  26.15%  40.58%  57.04%
mean ranking  0.416512  2746.068359  32.63%  47.10%  62.16%
